In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_excel('data/Bolsonaro + Lula - Validado.xlsx')

In [3]:
def day_for_flaot(x):
  if(isinstance(x, datetime)):
    x = float(f"{x.day}.{x.month}")
  return x

In [4]:
df['Duracao'] = df['Duracao'].apply(day_for_flaot)
df['Duracao'] = pd.to_numeric(df['Duracao'], errors='coerce', downcast='signed')

In [5]:
df['Duracao']

0      57.93
1      17.30
2      17.13
3      52.54
4      34.48
       ...  
564    51.34
565    15.20
566    85.10
567    12.93
568    49.19
Name: Duracao, Length: 569, dtype: float64

# Ao executar tudo será criado dois arquivos de dataset com os nomes 'file'_dataset.csv 'file'_dataset.xlsx

In [6]:
#file = 'Amazon'
file = 'Google'

In [7]:
if file.title() == 'Google':
  data = pd.read_csv('data/GoogleVision.csv')
  text = data.loc[data['Subclass']=='text'][['ID', 'Class']]
  text.rename({'Class': 'Text_Post'}, inplace=True, axis=1)
  data = data.loc[data['Subclass'] != 'text']
else:
  data = pd.read_csv('data/result_not_duplicates_oficial.csv')
  data.rename({'porcentagem': 'Percent', 'Nome': 'Class'}, axis=1, inplace=True)
  text = pd.DataFrame({'ID': [], 'Text_Post': []})

# Filtro de dados validados

In [8]:
data = data.loc[data['ID'].isin(df['ID'])]
text = text.loc[text['ID'].isin(df['ID'])]
data['ID'].unique().size

569

# Filtro de quartil

In [9]:
filter_percent = data['Percent'].quantile(.25)
filter_percent

0.74

In [10]:
print(f'''Quantidade Total: {data["ID"].count()}
      \nQuantidade esperado: {data["ID"].count()*.75}''')
data = data[data['Percent'] >= filter_percent]
print(f"Quantidade retornado: {data['ID'].count()}")

Quantidade Total: 7221
      
Quantidade esperado: 5415.75
Quantidade retornado: 5591


In [27]:
data.sort_values('Percent', ascending=False, inplace=True)

In [28]:
data[data['ID'] == 7150280209315417349]

,ID,Class,Percent,Subclass
9977,7150280209315417349,Nose,0.98,item
9978,7150280209315417349,Forehead,0.98,item
9973,7150280209315417349,Cheek,0.98,item
9979,7150280209315417349,Chin,0.97,item
9982,7150280209315417349,Jaw,0.88,item
9983,7150280209315417349,Microphone,0.86,item
9976,7150280209315417349,Person,0.84,object
9981,7150280209315417349,Hat,0.83,item
9984,7150280209315417349,Public address system,0.80,item
9972,7150280209315417349,Clothing,0.78,object


# DataFrame com ID, Class e o texto do Post

In [29]:
new_data = data.groupby('ID')['Class'].apply(' '.join).reset_index()
new_data = new_data.merge(text, how='outer', on='ID')
new_data

,ID,Class,Text_Post
0,7114971700365692165,Face Microphone Forehead Hair Head Public addr...,que alguém chegasse_quase sem comitiva_PRESIDE...
1,7115033431473474822,Nose Forehead Chin Mouth Beard Jaw Dress shirt...,O futuro do país!
2,7115050482179050758,Joy Forehead Smile Facial expression Hat Cap H...,"MAIS DE_4,2 MILHÕES_DE EMPREGOS GERADOS_ENTRE ..."
3,7115120078982630661,Exposed World Organism Font T-shirt Adaptation...,CAMPO GRANDE/MS (30/06/22)_> Entrega do Reside...
4,7115161088219565317,Chin Microphone Tie Person Dress shirt Coat Sl...,"O MEU ESTÔMAGO_TÁ SEMPRE RONCANDO,_PRESIDE_JAI..."
...,...,...,...
564,7160017587160485125,Blurred Hat Photograph Motor vehicle Motorcycl...,29/10_BELO HORIZONTE-MG
565,7160037888611126534,Shirt World Sleeve Person Yellow Plant Top T-s...,755_PROPAGANDA ELEITORAL-COLIGAÇÃO BRASIL DA E...
566,7160041692911930630,Hat Microphone Person World Gesture Font Spoke...,QUER_dad_MADOR_LULA 13_PRESIDENTE_ALCKMIN VICE...
567,7160077729751207174,Joy Forehead Smile Beard Tie Facial hair Colla...,LULA_MEU PRESIDENTE


In [22]:
new_data[new_data['ID'] == 7150280209315417349]

,ID,Class,Text_Post,Duracao,Plays,Comentarios,Compart.,Texto,Curtidas,Perfil,...,Classifier_Comentarios,Classifier_Compart.,Classifier_Curtidas,domingo,quarta-feira,quinta-feira,segunda-feira,sexta-feira,sábado,terça-feira
429,7150280209315417349,Clothing Cheek Person Nose Forehead Chin T-shi...,"NÓS VENCEMOS A MENTIRA_NO DIA DE HOJE,",56.03,947100,8712,2774,- O mundo atravessa por problemas há mais de 3...,103000,0,...,Viral,Bombou,Viral,0,0,0,1,0,0,0


In [13]:
df.columns

Index(['Unnamed: 0', 'DataColeta', 'Perfil', 'DataPost', 'DiaDaSemana',
       'Plays', 'Curtidas', 'Comentarios', 'Compart.', 'Texto', 'LinkFoto',
       'LinkVideo', 'LinkPost', 'ID', 'Duracao', 'Duração Classificada',
       'Retórica Aristotélica', 'Dispositivo Retórico', 'Tipo de conteúdo',
       'Abordagem', 'Tonalidade', 'Main character', 'Texto / Hashtag'],
      dtype='object')

# Existem algumas colunas que podem vazar informação tome cuidado com elas

In [14]:
columns = [
  'ID',
  'Perfil',
  #'DataColeta',
  #'DataPost',
  #'LinkFoto',
  #'LinkVideo',             Links não são pertinentes
  #'LinkPost',
  'Duracao',
  #'Duração Classificada',
  #'Retórica Aristotélica',
  #'Dispositivo Retórico',
  #'Tipo de conteúdo',      Classificações manual
  #'Abordagem',
  #'Tonalidade',
  #'Main character',
  #'Texto / Hashtag',
  'DiaDaSemana',           #
  'Plays',                 #
  'Comentarios',           # Colunas que podem causar vazamento das informações
  'Compart.',              #
  'Texto',                 #
  'Curtidas',
]

In [15]:
df = df[columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           569 non-null    int64  
 1   Perfil       569 non-null    object 
 2   Duracao      569 non-null    float64
 3   DiaDaSemana  569 non-null    object 
 4   Plays        569 non-null    int64  
 5   Comentarios  569 non-null    int64  
 6   Compart.     569 non-null    int64  
 7   Texto        569 non-null    object 
 8   Curtidas     569 non-null    int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 40.1+ KB


## OBS na coluna perfil os valores repesentam
- 0 == bolsonaromessiasjair
- 1 == lulaoficial 

In [16]:
df = pd.get_dummies(df, columns=['Perfil'], prefix='', prefix_sep='', drop_first=True, dtype='int')
df.rename({'lulaoficial': 'Perfil'}, inplace=True, axis=1)

# Classifique de acordo com o quartil de cada variável de engajamento
# Segue a ordem conforme a importância
* ### 1 - Viral
* ### 2 - Bombou
* ### 3 - Mediano
* ###  4 - Flopou

In [17]:
def create_class_quartis(label: str):
  values = [
    df[label].quantile(.25),
    df[label].quantile(.5),
    df[label].quantile(.75),
    ]
  def create_class(x):
    if x < values[0]:
      return f"Flopou"
    if values[0] <= x < values[1]:
      return f"Mediano"
    if values[1] <= x < values[2]:
      return f"Bombou"
    if values[2] <= x:
      return f"Viral"
  
  print(f'Classes para {label}')
  print(f"Classe 1: 0 <= x < {values[0]}")
  print(f"Classe 2: {values[0]} <= x < {values[1]}")
  print(f"Classe 3: {values[1]} <= x < {values[2]}")
  print(f"Classe 4: {values[2]} <= x < ∞")
  return df[label].apply(create_class)
  

In [18]:
classes = ['Plays', 'Comentarios', 'Compart.', 'Curtidas',]
for label in classes:
  df['Classifier_'+label] = create_class_quartis(label)

Classes para Plays
Classe 1: 0 <= x < 125200.0
Classe 2: 125200.0 <= x < 259900.0
Classe 3: 259900.0 <= x < 671700.0
Classe 4: 671700.0 <= x < ∞
Classes para Comentarios
Classe 1: 0 <= x < 387.0
Classe 2: 387.0 <= x < 869.0
Classe 3: 869.0 <= x < 2547.0
Classe 4: 2547.0 <= x < ∞
Classes para Compart.
Classe 1: 0 <= x < 408.0
Classe 2: 408.0 <= x < 1166.0
Classe 3: 1166.0 <= x < 4048.0
Classe 4: 4048.0 <= x < ∞
Classes para Curtidas
Classe 1: 0 <= x < 10500.0
Classe 2: 10500.0 <= x < 24500.0
Classe 3: 24500.0 <= x < 72200.0
Classe 4: 72200.0 <= x < ∞


In [19]:
new_data = new_data.merge(df, on='ID', how='outer')

In [20]:
new_data = pd.get_dummies(new_data, columns=['DiaDaSemana'], dtype='int', prefix='', prefix_sep='')

In [21]:
#new_data.to_csv(f'data/{file}_dataset.csv')
#new_data.to_excel(f'data/{file}_dataset.xlsx')